In [ ]:
# Adjustable Privacy - model_number_and_lambda_selection.ipynb
# - Uses image datasets (CelebA).
# - This code obfuscate test set by all model numbers (of obfuscator) without g and f; then evaluate two trained machine on them (one infer private attribute (gender) and another infer utility attribute)
# - reports the result of impact of choosing model number by ploting (saves result and plot)
# - Also after selection of a model, obfuscate testset by this model considering g and by different lambdas and evaluate two mentioned machine on them.
# - reports the result of impact of choosing lambda by ploting (saves result and plot)
# - It can load previously saved results.
# - You can manage notebook parameters in parser block

In [ ]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
from torchvision import datasets, transforms, models
import torchvision.utils as vutils
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools
import random
import shutil
from zipfile import ZipFile
import os
from tqdm import tqdm
from collections import OrderedDict
import time
from math import floor
import argparse

In [ ]:
!sudo apt-get update
!sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended texlive texlive-fonts-recommended
import os
from matplotlib.pyplot import text
matplotlib.style.use('default')
from matplotlib import rc

rc('font', **{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [ ]:
# Parser
parser = argparse.ArgumentParser(description='Adjustable Privacy '
                                 + 'Uses image dataset (CelebA). '
                                 + 'This code obfuscate test set by all model numbers (of obfuscator) without g and f; then evaluate two trained machine on them (one infer private attribute (gender) and another infer utility attribute). '
                                 + 'reports the result of impact of choosing model number by plotting (saves result and plot) '
                                 + 'Also after selection of a model, obfuscate testset by this model considering g and by different lambdas and evaluate two mentioned machine on them. '
                                 + 'reports the result of impact of choosing lambda by plotting (saves result and plot) '
                                 + 'It can load previously saved results. ')

parser.add_argument('--obf_model_number', type=int, required=True, help = 'The epoch number of desired obfuscator model (model number)')
parser.add_argument('--utl_model_number', type=int, required=True, help = 'The epoch number of desired trained model (infer target label - smiling, open mouth, and highcheekbone)')
parser.add_argument('--adv_model_number', type=int, required=True, help = 'The epoch number of desired trained model (infer gender)')
parser.add_argument('--target_index', type=int, required=True, help = 'smiling(31), open mouth(21), and high cheekbone(19)')
parser.add_argument('--obf_load_path', type=str, required=True, help = 'Full path on your google drive to load obfuscator model from. Like "drive/MyDrive/adjustable-privacy/Models/CelebA-G-S-Obfuscator/"')
parser.add_argument('--utl_load_path', type=str, required=True, help = 'Full path on your google drive to load trained model (infer target label - smiling, open mouth, and high cheekbone) from. Like "drive/MyDrive/adjustable-privacy/Models/CelebA-S/"')
parser.add_argument('--adv_load_path', type=str, required=True, help = 'Full path on your google drive to load trained model (infer gender) from. Like "drive/MyDrive/adjustable-privacy/Models/CelebA-G/"')
parser.add_argument('--save_path', type=str, required=True, help = 'Full path on your google drive to save results. Like "drive/MyDrive/adjustable-privacy/Others/Results/"')
parser.add_argument('--download_dataset', default = False, help = 'Accepts "True" or "False". Download CelebA dataset or use CelebA shared directory.')
parser.add_argument('--dataset_path', type=str, default = "", help = '(if --download_dataset=False) Full path on your google drive to CelebA shared directory shortcut. Like "drive/MyDrive/adjustable-privacy/Datasets/CelebA/"')
parser.add_argument('--load_result_model_number', default = False, help = 'Accepts "True" or "False". load last saved result of model number experiment')
parser.add_argument('--last_saved_model_number', type=int, default = 200, help = '(In model number experiment) In case of resuming experiment use last saved number and in case of loading result, set to desired number.')
parser.add_argument('--whole_model_numbers', type=int, default = 200, help = '(In model number experiment) Number of whole model numbers. ')
parser.add_argument('--model_number_result_file_name', type=str, default = "", help = 'a short name specify model number experiment result like:"GS-modelnumber" ')
parser.add_argument('--load_result_lambda', default = False, help = 'Accepts "True" or "False". load last saved result of lambda experiment')
parser.add_argument('--lambda_result_file_name', type=str, default = "", help = 'a short name specify result of lambda experiment like:"GS-lambda-obf183" or ""GS-lambda-obf13""')

command_string = "--obf_model_number 183" \
" --utl_model_number 6" \
" --adv_model_number 9" \
" --target_index 20" \
" --obf_load_path drive/MyDrive/adjustable-privacy/Models/CelebA-G-S-Obfuscator/" \
" --utl_load_path drive/MyDrive/adjustable-privacy/Models/CelebA-S/" \
" --adv_load_path drive/MyDrive/adjustable-privacy/Models/CelebA-G/" \
" --save_path drive/MyDrive/adjustable-privacy/Others/Results/" \
" --download_dataset False" \
" --dataset_path drive/MyDrive/adjustable-privacy/Datasets/CelebA/" \
" --load_result_model_number True" \
" --last_saved_model_number 200" \
" --whole_model_numbers 200" \
" --model_number_result_file_name GS-modelnumber" \
" --load_result_lambda True" \
" --lambda_result_file_name GS-lambda-obf183"

args = parser.parse_args(command_string.split())

In [ ]:
# Hyper parameters:
manual_seed = 20
image_size = 64
use_whole_dataset = True
usage_percent = 1.00
celeba_male_index = 20
celeba_smiling_index = 31
celeba_mouth_open_index = 21
celeba_high_cheekbone_index = 19
using_index = args.target_index
adv_using_index = celeba_male_index
batch_size = 64 #64

files_not_ready = True
download_dataset = args.download_dataset=='True'
dataset_folder_path = args.dataset_path
data_dir = 'celeba'
saving_path = args.save_path

# creating modified dataset:
m_version = 'model-number-and-lambda-experiments'
on_fly_modified_dataset_saving_path = 'modifiedDatasets/' + m_version + '/'

p2r_model_number = args.obf_model_number
p2r_model_path = args.obf_load_path
utl_model_number = args.utl_model_number
utl_model_path = args.utl_load_path
adv_model_number = args.adv_model_number
adv_model_path = args.adv_load_path

use_g = True
g_eff_val = -10000
miu = 0
coef_for_var = 0

suffling_main_train_data = False
suffling_modified_train_data = False

# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Size of feature maps in encoder
nef = 64
# Size of feature maps in decoder
ndf = 64
# Number of channels in the training images. For color images this is 3
nc = 3

In [ ]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# getting dataset ready from shared directory
if not download_dataset:
    dataset_zip_path = dataset_folder_path + '/Img/img_align_celeba.zip'
    list_eval_partition_path = dataset_folder_path + '/Eval/list_eval_partition.txt'
    identity_celeba_path = dataset_folder_path + '/Anno/identity_CelebA.txt'
    list_attr_celeba_path = dataset_folder_path + '/Anno/list_attr_celeba.txt'
    list_bbox_celeba_path = dataset_folder_path + '/Anno/list_bbox_celeba.txt'
    list_landmarks_align_celeba_path = dataset_folder_path + '/Anno/list_landmarks_align_celeba.txt'

    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_zip_path, data_dir + r'/img_align_celeba.zip')
    shutil.copyfile(list_eval_partition_path, data_dir + r'/list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, data_dir + r'/identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, data_dir + r'/list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, data_dir + r'/list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, data_dir + r'/list_landmarks_align_celeba.txt')

    try:
        shutil.rmtree(data_dir + r'/img_align_celeba')
        print("old unzipped directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

    archive = data_dir + r'/img_align_celeba.zip'
    with ZipFile(archive, 'r') as zip:
       zip.extractall(data_dir)

In [ ]:
# make saving path dir
try:
    os.makedirs(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

In [ ]:
# Function - use some percent of data
def shorten_dataset(dataset, usage_percent=1.0):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed))
  return used_dataset

In [ ]:
# Define transforms
test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
# Load Datas
test_set = datasets.CelebA(root='', download=download_dataset, split='test', target_type=["attr", "identity"], transform=test_transforms)
# shorten Dataset
if not use_whole_dataset:
  test_set = shorten_dataset(test_set, usage_percent)
# DataLoader
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers, drop_last=True)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
# Encoder Model
class Encoder(nn.Module):
    def __init__(self, ngpu):
        super(Encoder, self).__init__()
        self.ngpu = ngpu

        # input is nc x 64 x 64
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # split features: 1024 -> 1022 + 2
        # first classifier:
        self.fllc_main_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_main_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_main_features1 = nn.Dropout(p=0.5)
        self.fllc_main_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_main_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_main_features2 = nn.Dropout(p=0.5)
        self.fllc_main_features3 = nn.Linear(nef*4, nef)
        self.actv_main_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_main_features4 = nn.Linear(nef, 2)
        self.actv_main_features4 = nn.LogSoftmax(dim=1)
        # other features
        self.fllc_other_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_other_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_other_features1 = nn.Dropout(p=0.5)
        self.fllc_other_features2 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_other_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_other_features2 = nn.Dropout(p=0.5)
        self.fllc_other_features3 = nn.Linear(nef*1*4*4, nef*1*4*4 - 2)
        self.actv_other_features3 = nn.LeakyReLU(0.2, inplace=True)
        # aggregate features for output

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # first classifier:
        y1 = self.fllc_main_features1(x)
        y1 = self.actv_main_features1(y1)
        y1 = self.dropout_main_features1(y1)
        y1 = self.fllc_main_features2(y1)
        y1 = self.actv_main_features2(y1)
        y1 = self.dropout_main_features2(y1)
        y1 = self.fllc_main_features3(y1)
        y1 = self.actv_main_features3(y1)
        y1 = self.fllc_main_features4(y1)
        y1 = self.actv_main_features4(y1)
        # other features
        y3 = self.fllc_other_features1(x)
        y3 = self.actv_other_features1(y3)
        y3 = self.dropout_other_features1(y3)
        y3 = self.fllc_other_features2(y3)
        y3 = self.actv_other_features2(y3)
        y3 = self.dropout_other_features2(y3)
        y3 = self.fllc_other_features3(y3)
        y3 = self.actv_other_features3(y3)
        return y1, y3

In [ ]:
# Decoder Model
class Decoder(nn.Module):
    def __init__(self, ngpu):
        super(Decoder, self).__init__()
        self.ngpu = ngpu

        # input size is 1024
        self.fllc6 = nn.Linear(nef*1*4*4, ndf*2*4*4)
        self.actv6 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 2048
        # shaping would be here: 2048 -> ndf*2 x 4 x 4
        # state size. (ndf*2) x 4 x 4
        self.cnvt7 = nn.ConvTranspose2d( ndf*2, ndf, 4, 2, 1, bias=False)
        self.bnor7 = nn.BatchNorm2d(ndf)
        self.actv7 = nn.ReLU(True)
        # state size. (ndf) x 8 x 8
        self.cnvt8 = nn.ConvTranspose2d(ndf, ndf, 4, 2, 1, bias=False)
        self.bnor8 = nn.BatchNorm2d(ndf)
        self.actv8 = nn.ReLU(True)
        # state size. (ndf) x 16 x 16
        self.cnvt9 = nn.ConvTranspose2d( ndf, ndf, 4, 2, 1, bias=False)
        self.bnor9 = nn.BatchNorm2d(ndf)
        self.actv9 = nn.ReLU(True)
        # state size. (ndf) x 32 x 32
        self.cnvt10 = nn.ConvTranspose2d( ndf, nc, 4, 2, 1, bias=False)
        self.actv10 = nn.Sigmoid()
        # state size. (nc) x 64 x 64

    def forward(self, x):
        x = self.fllc6(x)
        x = self.actv6(x)
        x = x.view(batch_size, ndf*2, 4, 4)
        x = self.cnvt7(x)
        x = self.bnor7(x)
        x = self.actv7(x)
        x = self.cnvt8(x)
        x = self.bnor8(x)
        x = self.actv8(x)
        x = self.cnvt9(x)
        x = self.bnor9(x)
        x = self.actv9(x)
        x = self.cnvt10(x)
        x = self.actv10(x)
        return x

In [ ]:
# AE Model + Noise
class AEModel(nn.Module):
    def __init__(self, ngpu, mode='train', miu=0, coef_for_var=0, g_eff_val=-10000):
        super(AEModel, self).__init__()
        self.ngpu = ngpu
        self.g_eff_val = g_eff_val
        self.miu = miu
        self.coef_for_var = coef_for_var
        self.mode = mode
        self.encoder = Encoder(ngpu).to(device)
        self.decoder = Decoder(ngpu).to(device)

    def tune_noise(self, miu=0, coef_for_var=0, g_eff_val=-10000):
        self.miu = miu
        self.coef_for_var = coef_for_var
        self.g_eff_val = g_eff_val

    def change_mode(self, mode='train'):
        self.mode = mode

    def add_noise(self, nodes):
      with torch.no_grad():
        var = (self.coef_for_var) * (torch.mean(nodes).item())
        noise = self.miu + (var) * torch.randn(nodes.size())
        noise = noise.to(device)
        nodes.add_(noise)
        return nodes

    def change_lbl(self, nodes, lbls):
      with torch.no_grad():
        lbls[lbls == 0] = self.g_eff_val
        lbls[lbls == 1] = 0
        nodes = lbls
        return nodes

    def forward(self, x, y1_real_lbl=[]):
        y1, y3 = self.encoder(x)
        if self.mode=='use':
            if use_g:
              y1 = self.change_lbl(y1, y1_real_lbl)
            y3 = self.add_noise(y3)
        y = torch.cat((y1, y3), 1)
        x = self.decoder(y)
        return x, y1

In [ ]:
# Utilizer Model
class UtlModel(nn.Module):
    def __init__(self, ngpu):
        super(UtlModel, self).__init__()
        self.ngpu = ngpu

        # input is nc x 64 x 64
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # classifier:
        self.fllc_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features1 = nn.Dropout(p=0.5)
        self.fllc_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features2 = nn.Dropout(p=0.5)
        self.fllc_features3 = nn.Linear(nef*4, nef)
        self.actv_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_features4 = nn.Linear(nef, 2)
        self.actv_features4 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # classifier: 
        y1 = self.fllc_features1(x)
        y1 = self.actv_features1(y1)
        y1 = self.dropout_features1(y1)
        y1 = self.fllc_features2(y1)
        y1 = self.actv_features2(y1)
        y1 = self.dropout_features2(y1)
        y1 = self.fllc_features3(y1)
        y1 = self.actv_features3(y1)
        y1 = self.fllc_features4(y1)
        y1 = self.actv_features4(y1)
        return y1

In [ ]:
# For Adversary
# Adversary Model 
class AdvModel(nn.Module):
    def __init__(self, ngpu):
        super(AdvModel, self).__init__()
        self.ngpu = ngpu
        
        # input is nc x 64 x 64 
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # classifier: 
        self.fllc_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features1 = nn.Dropout(p=0.5)
        self.fllc_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features2 = nn.Dropout(p=0.5)
        self.fllc_features3 = nn.Linear(nef*4, nef)
        self.actv_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_features4 = nn.Linear(nef, 2)
        self.actv_features4 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # classifier: 
        y1 = self.fllc_features1(x)
        y1 = self.actv_features1(y1)
        y1 = self.dropout_features1(y1)
        y1 = self.fllc_features2(y1)
        y1 = self.actv_features2(y1)
        y1 = self.dropout_features2(y1)
        y1 = self.fllc_features3(y1)
        y1 = self.actv_features3(y1)
        y1 = self.fllc_features4(y1)
        y1 = self.actv_features4(y1)
        return y1

In [ ]:
# Create 
netAE = AEModel(ngpu).to(device)
adversaryModel = AdvModel(ngpu).to(device)
utilizerModel = UtlModel(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netAE = nn.DataParallel(netAE, list(range(ngpu)))
    adversaryModel = nn.DataParallel(adversaryModel, list(range(ngpu)))
    utilizerModel = nn.DataParallel(utilizerModel, list(range(ngpu)))

In [ ]:
#@title
# Function - Save:
def save_data(saving_path, name, res):
  checkpoint = {'res': res}
  torch.save(checkpoint, saving_path + name + '.pth')
  return True

In [ ]:
def load_data(saving_path, name):
  checkpoint = torch.load(saving_path + name + '.pth', map_location=device)
  res = checkpoint['res']
  return res

In [ ]:
#@title
# Function - Load:
def load_model(saving_path, name, number, model, device):

  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [ ]:
# Load Last Checkpoint:

def load_models():
  ae_load = load_model(p2r_model_path, 'ae', p2r_model_number, netAE, device)
  adv_load = load_model(adv_model_path, 'ins', adv_model_number, adversaryModel, device)
  utl_load = load_model(utl_model_path, 'ins', utl_model_number, utilizerModel, device)
  return ae_load, adv_load, utl_load

In [ ]:
def extract_targets(labels):
    male_labels = labels[0][:, using_index]
    female_labels = torch.add(1, -male_labels)
    gender_target = torch.cat((female_labels.view(batch_size,1), male_labels.view(batch_size,1)), 1).float()
    gender_target = gender_target.to(device)
    return gender_target

In [ ]:
m_celeba_dir = on_fly_modified_dataset_saving_path + 'celeba/'
img_celeba_dir = on_fly_modified_dataset_saving_path + 'celeba/img_align_celeba/'

In [ ]:
# Create new dataset folder:
def create_modified_dataset_folder():
  try:
      os.mkdir('modifiedDatasets')
  except OSError as e:
      print("Error: %s" % (e.strerror))
  try:
      os.mkdir(on_fly_modified_dataset_saving_path)
  except OSError as e:
      print("Error: %s" % (e.strerror))
  try:
      os.mkdir(m_celeba_dir)
  except OSError as e:
      print("Error: %s" % (e.strerror))
  try:
      os.mkdir(img_celeba_dir)
  except OSError as e:
      print("Error: %s" % (e.strerror))

  try:
      shutil.copyfile(list_eval_partition_path, m_celeba_dir + r'list_eval_partition.txt')
      shutil.copyfile(identity_celeba_path, m_celeba_dir + r'identity_CelebA.txt')
      shutil.copyfile(list_attr_celeba_path, m_celeba_dir + r'list_attr_celeba.txt')
      shutil.copyfile(list_bbox_celeba_path, m_celeba_dir + r'list_bbox_celeba.txt')
      shutil.copyfile(list_landmarks_align_celeba_path, m_celeba_dir + r'list_landmarks_align_celeba.txt')
      print("label files copied successfully")
  except OSError as e:
      print("Error: %s" % (e.strerror))

In [ ]:
def delete_modified_dataset_folder():
    try:
        shutil.rmtree(on_fly_modified_dataset_saving_path)
        print("old modified_dataset_folder removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

In [ ]:
# Function save image:
from torchvision.utils import save_image

def store_single_disk(image, file_name):
    save_image(image, img_celeba_dir + file_name)

In [ ]:
def fix_getting_filename_problem(my_loader,i,j,batch_size):
    if use_whole_dataset:
        return my_loader.dataset.filename[i*batch_size + j]
    else:
        return my_loader.dataset.dataset.filename[i*batch_size + j]

In [ ]:
def convert_dataset(my_loader):
    prog_bar = tqdm(enumerate(my_loader), total=len(my_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            gender_target = extract_targets(labels)
            output, y1 = netAE.forward(inputs, gender_target)
            for j in range(batch_size):
                store_single_disk(output[j,:,:,:], fix_getting_filename_problem(my_loader,i,j,batch_size))

In [ ]:
# using model to obfuscate dataset
def convert_testset():
  netAE.change_mode('use')
  netAE.eval()
  netAE.tune_noise(miu, coef_for_var, g_eff_val)
  print("\nConverting test images...")
  convert_dataset(test_loader)

In [ ]:
# load modified dataset:
def created_modified_dataloader():
  m_transforms = transforms.Compose([transforms.ToTensor()])
  m_test_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='test', target_type=["attr", "identity"], transform=m_transforms)
  m_test_loader = torch.utils.data.DataLoader(m_test_set, batch_size=batch_size, num_workers=workers, drop_last=True)
  return m_test_loader

In [ ]:
# Calc Accuracy
def calcAccuracy(model, test_loader, using_index):
    model.to(device)
    model.eval()
    y1_accuracy = 0
    prog_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            young_target = labels[0][:, using_index]
            young_target = young_target.to(device)
            output = model(inputs)
            ps_y1 = torch.exp(output)
            top_p_y1, top_class_y1 = ps_y1.topk(1, dim=1)
            equals_y1 = top_class_y1 == young_target.view(*top_class_y1.shape)
            acc_y1 = equals_y1.sum().item()
            y1_accuracy += (acc_y1 / len(equals_y1))            
    y1_accuracy = y1_accuracy / len(test_loader)
    return y1_accuracy

In [ ]:
# Test on obfuscated data
def test_on_obfuscated_data(model, loader, index):
  model.to(device)
  acc = calcAccuracy(model, loader, index)
  return acc

In [ ]:
# Search for best p2r model number - loop (all values)

load_saved_data = args.load_result_model_number=='True'
last_saved_number = args.last_saved_model_number

save_every_i = 5
whole_epochs = args.whole_model_numbers
if load_saved_data:
  whole_epochs = args.whole_model_numbers - last_saved_number
p2r_epoch_range = range(whole_epochs)

use_g = False
g_eff_val = -1
miu = 0
coef_for_var = 0

adv_acc=[]
utl_acc=[]

if load_saved_data:
  res = load_data(saving_path, args.model_number_result_file_name + str(last_saved_number))
  adv_acc = res['adv_acc']
  utl_acc = res['utl_acc']
  
if not load_saved_data:
  for i in p2r_epoch_range:
    if load_saved_data:
      i = i + (last_saved_number+1) 
    print(f"Epoch {i}/{whole_epochs}: ")
    p2r_model_number = i
    ae_load, adv_load, utl_load = load_models()
    delete_modified_dataset_folder()
    create_modified_dataset_folder()
    convert_testset()
    m_test_loader = created_modified_dataloader()
    adv_accuracy = test_on_obfuscated_data(adversaryModel, m_test_loader, adv_using_index)
    adv_acc.append(adv_accuracy)
    utilizerModel.to(device)
    utl_accuracy = test_on_obfuscated_data(utilizerModel, m_test_loader, using_index)
    utl_acc.append(utl_accuracy)
    print(f"\n Adversary Accuracy on Testset: {adv_accuracy:.6f}")
    print(f"\n Utilizer Accuracy on Testset: {utl_accuracy:.6f}")
    if i % save_every_i == 0:
      res = {'adv_acc': adv_acc,'utl_acc': utl_acc };
      save_data(saving_path, args.model_number_result_file_name + str(i), res)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
matplotlib.style.use('default')
from matplotlib import rc

rc('font', **{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

print('Select obfuscator model number plot...')

fig = plt.figure(figsize=(5,3))
matplotlib.rcParams.update({'font.size': 10})
fig.tight_layout()
plt.plot(adv_acc, color='red', label='A Well-Trained Network for Gender Inference')
plt.plot(utl_acc, color='green', label='A Well-Trained Network for Smiling Inference')
plt.xlabel("Obfuscator Model Number (Training Epoch Number)")
plt.ylabel("Accuracy")
plt.xlim([0,200])
plt.legend()
plt.grid(color = 'gray', linestyle = 'dotted', linewidth = 0.5)
plt.savefig(saving_path + args.model_number_result_file_name + "_plot.svg", bbox_inches = 'tight')
plt.savefig(saving_path + args.model_number_result_file_name + "_plot.png", bbox_inches = 'tight')
plt.savefig(saving_path + args.model_number_result_file_name + "_plot.eps", bbox_inches = 'tight', format='eps')
plt.show()

In [ ]:
# Search for best g power (g_eff_value)
load_saved_data = args.load_result_lambda=='True'
g_eff_vals = [-1, -500, -1000, -2000, -3000, -4000, -5000, -10000]

use_g = True
miu = 0
coef_for_var = 0
p2r_model_number = args.obf_model_number  # 183 or 13
adv_acc_p2rnum1 = []
utl_acc_p2rnum1 = []

if load_saved_data:
  res_p2rnum1 = load_data(saving_path, args.lambda_result_file_name) # 183 -> 13
  adv_acc_p2rnum1 = res_p2rnum1['adv_acc']
  utl_acc_p2rnum1 = res_p2rnum1['utl_acc']
else:  
  for g_eff_val in g_eff_vals:
    ae_load, adv_load, utl_load = load_models()
    delete_modified_dataset_folder()
    create_modified_dataset_folder()
    convert_testset()
    m_test_loader = created_modified_dataloader()
    adv_accuracy = test_on_obfuscated_data(adversaryModel, m_test_loader, adv_using_index)
    utilizerModel.to(device)
    utl_accuracy = test_on_obfuscated_data(utilizerModel, m_test_loader, using_index)
    adv_acc_p2rnum1.append(adv_accuracy)
    utl_acc_p2rnum1.append(utl_accuracy)
    print(f"\n Adversary Accuracy on Testset: {adv_accuracy:.6f}")
    print(f"\n Utilizer Accuracy on Testset: {utl_accuracy:.6f}")
  res_p2rnum1 = {'adv_acc': adv_acc_p2rnum1,'utl_acc': utl_acc_p2rnum1 };
  save_data(saving_path, args.lambda_result_file_name, res_p2rnum1) # 183 -> 13

In [ ]:
%matplotlib inline
matplotlib.style.use('default')
from matplotlib import rc

rc('font', **{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
print('Select lambda plot...')

fig = plt.figure(figsize=(5,3))
matplotlib.rcParams.update({'font.size': 10})
fig.tight_layout()
plt.title("lambda selection for model number" + str(args.obf_model_number)) # 183 -> 13
plt.plot(g_eff_vals, adv_acc_p2rnum1, color='red', label='A Well-Trained Network for Gender Inference', marker='^')
plt.plot(g_eff_vals, utl_acc_p2rnum1, color='green', label='A Well-Trained Network for Smiling Inference', marker='^')

line_lambda = -3000
line_index = 4
if args.obf_model_number == 13:
  line_lambda = -2000
  line_index = 3

plt.axvline(x=line_lambda, linewidth=1, color='blue', label='$\lambda=' + str(line_lambda) + '$') # -3000 or -2000
plt.plot([line_lambda], adv_acc_p2rnum1[line_index], color='red', marker='^') # -3000 or -2000 / 3 or 4
plt.plot([line_lambda], utl_acc_p2rnum1[line_index], color='green', marker='^') # -3000 or -2000 / 3 or 4
plt.xlabel(r'$\lambda$')
plt.ylabel("Accuracy")
plt.xlim([-10100,+100])
plt.legend(fontsize=8, loc=5, bbox_to_anchor=(0.706,0.5))
plt.grid(color = 'gray', linestyle = 'dotted', linewidth = 0.5)
plt.savefig(saving_path + args.lambda_result_file_name + "_plot.svg", bbox_inches = 'tight') # 183 -> 13
plt.savefig(saving_path + args.lambda_result_file_name + "_plot.png", bbox_inches = 'tight') # 183 -> 13
plt.savefig(saving_path + args.lambda_result_file_name + "_plot.eps", bbox_inches = 'tight', format='eps') # 183 -> 13
plt.show()